# Roma Satellite Imagery Analysis with OpenEO

## Overview

This notebook demonstrates how to retrieve and visualize satellite imagery of Rome using the OpenEO API. We'll use Sentinel-2 data to create a true-color RGB image of the Rome area.

In [74]:
# Import required packages

import openeo
from openeo.processes import process
from IPython.display import Image

## Connect to the OpenEO Backend

We connect to the OpenEO backend service and authenticate using OpenID Connect. This establishes a connection to the remote processing service where our satellite data is stored.

In [75]:
# Connect to the back-end

#connection = openeo.connect("https://openeo.ds.io").authenticate_oidc()
connection = openeo.connect(url="http://127.0.0.1:8081/").authenticate_oidc()

Authenticated using refresh token.


## Define Spatial Extent

We define the geographic boundaries of our area of interest using longitude and latitude coordinates.

In [76]:
# Rome, italy
spatial_extent_east = 12.453
spatial_extent_west = 12.283
spatial_extent_north = 41.995
spatial_extent_south = 41.890

## Load Sentinel-2 Data

We load Sentinel-2 satellite imagery for our defined area using the `load_collection_and_reduce` process. 
- We select the RGB bands (B04=Red, B03=Green, B02=Blue) for true-color visualization
- The temporal extent covers April to December 2022
- The "first" pixel selection method means we'll use the first available cloud-free pixel in the time range

In [77]:
load1 = connection.datacube_from_process(
    "load_collection_and_reduce",
    id="sentinel-2-l2a",
    bands=["/measurements/reflectance/r10m:b04", "/measurements/reflectance/r10m:b03", "/measurements/reflectance/r10m:b02"],
    properties={},
    spatial_extent={
        "east": spatial_extent_east,
        "north": spatial_extent_north,
        "south": spatial_extent_south,
        "west": spatial_extent_west,
    },
    temporal_extent=["2025-09-21T00:00:00Z", "2025-09-22T00:00:00Z"],
    pixel_selection="first"
)
load1 = connection.datacube_from_process(
    "load_zarr",
    url="s3://esa-zarr-sentinel-explorer-fra/tests-output/sentinel-2-l2a/S2B_MSIL2A_20250921T100029_N0511_R122_T33TTG_20250921T135752.zarr",
    spatial_extent={
        "east": spatial_extent_east,
        "north": spatial_extent_north,
        "south": spatial_extent_south,
        "west": spatial_extent_west,
    },
    options={
        "variables": ["/measurements/reflectance/r10m:b04", "/measurements/reflectance/r10m:b03", "/measurements/reflectance/r10m:b02"],
        "width": 512,
        "height": 512,
    }
)

## Scale Image Values

Raw satellite reflectance values need to be scaled to standard RGB display range (0-255). We define a processing function that:
1. Scales the values from their original range (0-10000) to 0-255
2. Applies truncation to ensure all values are valid integers within range

In [78]:
def process1(x, context = None):
    data1 = process("linear_scale_range", inputMax = 1, inputMin = 0, outputMax = 255, x = x)
    data2 = process("trunc", x = data1)
    return data2

## Apply Processing and Enhance Visual Appearance

We apply the scaling function to our data cube, then use a color formula to enhance the visual appearance of the image. The color formula applies:
- Gamma correction (RGB 1.5) to adjust brightness and contrast
- Sigmoidal contrast enhancement (RGB 10 0.3) to improve detail visibility
- Saturation adjustment to enhance color vibrancy

In [79]:
processed = load1.apply(process = process1)

color = processed.process(
    "color_formula",
    data=processed,
    formula="Gamma RGB 1.5 Sigmoidal RGB 6 0.3 Saturation 1",
)

## Save and Display Result

Finally, we save the processed image as a PNG file and display it. This shows the true-color satellite view of Rome and surrounding areas.

In [ ]:
save5 = color.save_result(format="GTiff")

print (save5.to_json())

# The process can be executed synchronously (see below), as batch job or as web service now
# result = connection.download(save5)
# # write to file
# with open("rome_20250921.tif", "wb") as f:
#     f.write(result)

{
  "process_graph": {
    "loadzarr1": {
      "process_id": "load_zarr",
      "arguments": {
        "options": {
          "variables": [
            "/measurements/reflectance/r10m:b04",
            "/measurements/reflectance/r10m:b03",
            "/measurements/reflectance/r10m:b02"
          ],
          "width": 512,
          "height": 512
        },
        "spatial_extent": {
          "east": 12.453,
          "north": 41.995,
          "south": 41.89,
          "west": 12.283
        },
        "url": "s3://esa-zarr-sentinel-explorer-fra/tests-output/sentinel-2-l2a/S2B_MSIL2A_20250921T100029_N0511_R122_T33TTG_20250921T135752.zarr"
      }
    },
    "apply1": {
      "process_id": "apply",
      "arguments": {
        "data": {
          "from_node": "loadzarr1"
        },
        "process": {
          "process_graph": {
            "linearscalerange1": {
              "process_id": "linear_scale_range",
              "arguments": {
                "inputMax": 1,
       

## Conclusion

This notebook demonstrates how to use OpenEO to access, process, and visualize satellite imagery for urban areas. The workflow can be extended to include additional analysis, such as:

- Time series analysis to observe changes over multiple dates
- Land cover classification to identify different urban features
- Spectral indices to analyze vegetation, water, or built-up areas
- Image segmentation to extract specific features